<a href="https://colab.research.google.com/github/SamuelOnyangoOmondi/Build-a-chatbot/blob/main/plas_tech_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install sentence-transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 3.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [6]:
# Import necessary libraries
import pandas as pd
import json
from sentence_transformers import SentenceTransformer, util

# Load the dataset
dataset_path = '/content/drive/MyDrive/Summmative/plas_tech_dataset (1).json'
with open(dataset_path, 'r') as file:
    data = json.load(file)

# Convert the dataset into a DataFrame
df = pd.DataFrame(data)


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [7]:
# Data Preprocessing
def preprocess_text(text):
    text = text.lower()  # Convert text to lowercase
    text = ''.join([char for char in text if char.isalnum() or char.isspace()])  # Remove special characters
    return text

# Apply preprocessing to the context and response columns
df['context'] = df['context'].apply(preprocess_text)
df['response'] = df['response'].apply(preprocess_text)


In [8]:
# Load the SentenceTransformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Compute embeddings for the context sentences in the dataset
context_embeddings = model.encode(df['context'].tolist(), convert_to_tensor=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
# Build the Chatbot Interface
def chatbot_response(user_input):
    user_input = preprocess_text(user_input)
    user_input_embedding = model.encode(user_input, convert_to_tensor=True)

    # Compute cosine similarities
    cosine_scores = util.pytorch_cos_sim(user_input_embedding, context_embeddings)[0]

    # Find the index of the most similar context
    most_similar_idx = cosine_scores.argmax().item()

    return df['response'].iloc[most_similar_idx]

# Interactive Chatbot
def interactive_chatbot():
    print("Welcome to the Plas-tech Chatbot! Type 'exit' to end the conversation.")
    while True:
        user_input = input("You: ")
        if user_input.lower() == 'exit':
            print("Chatbot: Goodbye!")
            break
        response = chatbot_response(user_input)
        print(f"Chatbot: {response}")

# Start the interactive chatbot
interactive_chatbot()


Welcome to the Plas-tech Chatbot! Type 'exit' to end the conversation.
You: What is Plas-tech's mission?
Chatbot: the mission of plastech is to revolutionize sustainable energy solutions by converting plastic waste into clean cooking gas through innovation and collaboration we aim to reduce environmental impact empower communities and foster a greener more sustainable future
You: Who are the founder's of Plas-tech?
Chatbot: the founders of plastech are robin kariuki ceo zubeda khalifa product manager abuosama said cto and samuel omondi tech  marketing lead
You: What is Plas-tech Energies limited?
Chatbot: plastech energies limited is an innovative company dedicated to revolutionizing the way we manage plastic waste by converting it into clean cooking gas our mission is to create a cleaner more sustainable world through cuttingedge technology and community engagement
You: What is Plas-tech's vision? 
Chatbot: plastech envisions a world where plastic waste is not a problem but a valuable